# Pipeline

1. Preprocessing
2. **Data Extraction**
3. Data Exploration
4. Model

This file performs data extraction. It generates several features and returns a .pkl file containing a dataframe with said features.

# Imports

In [1]:
import pandas as pd
import numpy as np
import datetime
from collections import defaultdict

# Data extraction into new Dataframe

In [65]:
# We don't need HomeID and AwayID here, but I put it in just for exploration
results = pd.read_pickle('preprocessed_results.pkl')
df = results[['HomeID', 'AwayID', 'FTHG', 'FTAG', 'FTR']].copy()
df

,HomeID,AwayID,FTHG,FTAG,FTR
0,0,15,3,0,1
1,1,12,1,0,1
2,2,16,0,0,0
3,3,14,6,0,1
4,4,11,2,2,0
...,...,...,...,...,...
13015,97,105,2,2,0
13016,111,106,2,0,1
13017,120,109,0,2,2
13018,122,107,2,2,0


In [76]:
df = pd.read_pickle('feature_frame.pkl')
df['Date'] = results['Date']

In [53]:
results.sample(3)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
11381,SP1,2015-03-14,119,106,1,2,2,0,2,A,...,5,4,14,12,8,5,3,3,0,1
8199,I1,2015-11-21,71,62,2,2,0,1,0,H,...,3,5,22,18,5,5,5,4,0,0
10835,SP1,2013-10-27,105,117,5,0,1,1,0,H,...,12,1,14,7,6,5,0,2,0,0


In [4]:
# "Constants"
suffixes = {'HomeID':'_home', 'AwayID':'_away'}    # suffix dict for the feature names

id_names = ['HomeID', 'AwayID']                    # list of ID columns for iteration purposes

# suffix that's appended to all league-wide features
league_suffix = '_league'

In [5]:
#slc = df['AwayID'] == 105
#df[slc][['FTR']].value_counts() / sum(slc)

# Implementing non-relational Features from the Hubáček et al. paper

__For the moment, the features are closely aligned to those implemented by Hubáček et al. That means that most features also get implemented twice, once for the home team and once for that away team. The naming convention for this approach is FEATURE_NAME_home/away.__

Example: <br>
A_WIN_PCT_home would be the historical away winning percentage of the team that plays the current game at home. <br>
H_WIN_PCT_home would be the historical home winning percentage of that same team.

That means the word after the last underscore indicates if the value of this feature represents the home team or the away team of the current(!) game.

The current approach is to prefer readability over performance which means that it can take long to add the features to the dataframe.

In [43]:
# TODO: evtl. die feature-erstellungen in Funktionen packen um sie an und aus togglen zu können
# Alle nicht-kursiven Features aus dem Hubacek Paper müssen doppelt implementiert werden, siehe oben.

## Historical strength

In [5]:
# Returns all games that were played by a specific team between date minus two years and date.
def get_historical_games(team_id, date):
    games_by_team = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] < date) & (results['Date'] > date - datetime.timedelta(730))]
    return games_by_team

In [20]:
get_historical_games(8, datetime.date(2016, 5, 20)).head(6)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1527,E1,2014-08-17,8,24,2,1,1,1,0,H,...,5,6,8,11,2,6,1,2,0,0
1539,E1,2014-08-25,18,8,3,1,1,1,0,H,...,4,3,13,7,6,7,1,1,0,0
1549,E1,2014-08-31,5,8,0,3,2,0,1,A,...,1,8,14,11,7,3,0,4,0,0
1553,E1,2014-09-13,8,0,0,1,2,0,1,A,...,1,1,9,10,7,6,2,1,0,0
1565,E1,2014-09-20,15,8,3,1,1,2,1,H,...,7,5,16,11,7,5,4,1,0,0
1574,E1,2014-09-27,8,12,1,1,0,0,0,D,...,8,5,8,9,8,2,2,1,0,0


### Winning and drawing percentages

In [22]:
# Add percentages to dataframe
df['H_WIN_PCT_home'] = 0
df['H_DRAW_PCT_home'] = 0
df['A_WIN_PCT_home'] = 0
df['A_DRAW_PCT_home'] = 0
df['H_WIN_PCT_away'] = 0
df['H_DRAW_PCT_away'] = 0
df['A_WIN_PCT_away'] = 0
df['A_DRAW_PCT_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]
        
        hist_games = get_historical_games(team_id, game_date)
        game_count = len(hist_games)

        # total counts of home and away games in the last two years
        home_game_count = len(hist_games[hist_games['HomeID'] == team_id])
        away_game_count = len(hist_games[hist_games['AwayID'] == team_id])

        home_games_won = len(hist_games[(hist_games['HomeID'] == team_id) & (hist_games['FTR'] == 1)])
        away_games_won = len(hist_games[(hist_games['AwayID'] == team_id) & (hist_games['FTR'] == 2)])

        home_games_drawn = len(hist_games[(hist_games['HomeID'] == team_id) & (hist_games['FTR'] == 0)])
        away_games_drawn = len(hist_games[(hist_games['AwayID'] == team_id) & (hist_games['FTR'] == 0)])

        if home_game_count > 0:         # TODO should the threshold be higher to only use significant enough data?
            df.loc[row, 'H_WIN_PCT' + suffix] = home_games_won / home_game_count
            df.loc[row, 'H_DRAW_PCT' + suffix] = home_games_drawn / home_game_count
        else:
            df.loc[row, 'H_WIN_PCT' + suffix] = 0          # TODO maybe choose some standard value like 0.25 instead of 0?
            df.loc[row, 'H_DRAW_PCT' + suffix] = 0         # Overall Median/Mean could also work.

        if away_game_count > 0:
            df.loc[row, 'A_WIN_PCT' + suffix] = away_games_won / away_game_count
            df.loc[row, 'A_DRAW_PCT' + suffix] = away_games_drawn / away_game_count
        else:
            df.loc[row, 'A_WIN_PCT' + suffix] = 0          
            df.loc[row, 'A_DRAW_PCT' + suffix] = 0 

### Goal averages and standard deviations

In [23]:
# Add goal averages and deviations to dataframe
df['H_GS_AVG_home'] = 0
df['H_GC_AVG_home'] = 0
df['A_GS_AVG_home'] = 0
df['A_GC_AVG_home'] = 0
df['H_GS_AVG_away'] = 0
df['H_GC_AVG_away'] = 0
df['A_GS_AVG_away'] = 0
df['A_GC_AVG_away'] = 0

df['H_GS_STD_home'] = 0
df['H_GC_STD_home'] = 0
df['A_GS_STD_home'] = 0
df['A_GC_STD_home'] = 0
df['H_GS_STD_away'] = 0
df['H_GC_STD_away'] = 0
df['A_GS_STD_away'] = 0
df['A_GC_STD_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]
        
        hist_games = get_historical_games(team_id, game_date)
        game_count = len(hist_games)
        
        # calculate averages
        df.loc[row, 'H_GS_AVG' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTHG'].mean()
        df.loc[row, 'H_GC_AVG' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTAG'].mean()
        df.loc[row, 'A_GS_AVG' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTAG'].mean()
        df.loc[row, 'A_GC_AVG' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTHG'].mean()
        
        # calculate standard deviations
        df.loc[row, 'H_GS_STD' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTHG'].std()
        df.loc[row, 'H_GC_STD' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTAG'].std()
        df.loc[row, 'A_GS_STD' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTAG'].std()
        df.loc[row, 'A_GC_STD' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTHG'].std()

In [24]:
df.sample(5)

,HomeID,AwayID,FTHG,FTAG,FTR,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,H_WIN_PCT_away,...,A_GS_AVG_away,A_GC_AVG_away,H_GS_STD_home,H_GC_STD_home,A_GS_STD_home,A_GC_STD_home,H_GS_STD_away,H_GC_STD_away,A_GS_STD_away,A_GC_STD_away
3103,12,16,3,0,1,0.578947,0.210526,0.131579,0.368421,0.333333,...,1.000000,2.666667,1.430465,1.194166,0.749110,1.398027,2.081666,0.577350,1.000000,0.577350
10956,105,104,4,0,1,0.717949,0.153846,0.500000,0.263158,0.605263,...,1.307692,1.769231,1.706541,0.965673,1.131471,0.996794,1.441362,1.007793,1.079811,1.494931
10249,99,103,3,1,1,0.562500,0.125000,0.218750,0.187500,0.333333,...,0.903226,1.935484,1.310242,1.247174,0.974990,1.334635,0.951474,1.000000,0.870051,1.547805
1237,15,0,0,0,0,0.434783,0.260870,0.166667,0.291667,0.210526,...,1.081081,1.702703,1.145536,1.136877,0.775532,1.372610,1.229093,1.032887,1.115008,1.613569
1061,21,5,1,2,2,0.411765,0.382353,0.235294,0.235294,0.589744,...,1.552632,1.289474,1.281988,1.158169,1.174597,1.397732,1.089144,1.122702,1.178578,1.271473


## Current form (data from the last five games)

In [11]:
# df_mode indicates whether to return the df dataframe or the results dataframe
def get_last_n_games(team_id, date, n=5, df_mode=False):
    if df_mode == False:
        games_by_team = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] < date)].sort_values(by='Date', ascending=False).head(n)
    else:
        games_by_team = df[((df['HomeID'] == team_id) | (df['AwayID'] == team_id))
                           & (df['Date'] < date)].sort_values(by='Date', ascending=False).head(n)
        
    return games_by_team

In [17]:
get_last_n_games(19, datetime.date(2011, 6, 1))

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
374,E1,2011-05-22,9,19,4,2,1,1,1,D,...,11,7,8,5,8,8,2,1,0,0
361,E1,2011-05-14,19,2,4,3,1,3,2,H,...,8,9,16,12,5,6,2,4,0,0
353,E1,2011-05-07,5,19,1,1,0,0,0,D,...,11,7,5,13,7,6,3,2,0,0
340,E1,2011-04-30,19,13,0,0,0,0,0,D,...,6,6,10,11,4,8,0,0,0,0
328,E1,2011-04-23,19,17,1,1,0,1,1,D,...,7,6,17,8,4,4,1,1,0,0


### Winning and drawing percentages

In [ ]:
df['WIN_PCT_home'] = 0
df['WIN_PCT_away'] = 0
df['DRAW_PCT_home'] = 0
df['DRAW_PCT_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        recent = get_last_n_games(team_id, game_date)
        #print(len(recent))
        # game_count could be less than five if not enough data is available; that's the only reason for this line
        game_count = len(recent)
        
        games_won = len(recent[(recent['HomeID'] == team_id) & (recent['FTR'] == 1)
                    | (recent['AwayID'] == team_id) & (recent['FTR'] == 2)])
        
        games_drawn = len(recent[recent['FTR'] == 0])
        
        suffix = suffixes[id_name]
        
        if game_count > 0:
            df.loc[row, 'WIN_PCT' + suffix] = games_won / game_count
            df.loc[row, 'DRAW_PCT' + suffix] = games_drawn / game_count
        else:
            df.loc[row, 'WIN_PCT' + suffix] = 0
            df.loc[row, 'DRAW_PCT' + suffix] = 0
            

### Goal averages and standard deviations

In [27]:
df['GS_AVG_home'] = 0
df['GC_AVG_home'] = 0
df['GS_AVG_away'] = 0
df['GC_AVG_away'] = 0

df['GS_STD_home'] = 0
df['GC_STD_home'] = 0
df['GS_STD_away'] = 0
df['GC_STD_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]
        
        recent = get_last_n_games(team_id, game_date)
        game_count = len(recent)
        
        if game_count > 0:    # TODO gucken ob man vielleicht höheren threshold nehmen will und dann einfach mean assigned
            df.loc[row, 'GS_AVG' + suffix] = ((recent[recent['HomeID'] == team_id]['FTHG'].sum())
                                                + (recent[recent['AwayID'] == team_id]['FTAG'].sum())) / game_count

            df.loc[row, 'GC_AVG' + suffix] = ((recent[recent['HomeID'] == team_id]['FTAG'].sum())
                                                + (recent[recent['AwayID'] == team_id]['FTHG'].sum())) / game_count
            
            df.loc[row, 'GS_STD' + suffix] = np.std(np.append(recent[recent['HomeID'] == team_id]['FTHG'].tolist(),
                                                recent[recent['AwayID'] == team_id]['FTAG'].tolist()))
        
            df.loc[row, 'GC_STD' + suffix] = np.std(np.append(recent[recent['HomeID'] == team_id]['FTAG'].tolist(),
                                                recent[recent['AwayID'] == team_id]['FTHG'].tolist()))
            
        else:
            df.loc[row, 'GS_AVG' + suffix] = 0 
            df.loc[row, 'GC_AVG' + suffix] = 0
            df.loc[row, 'GS_STD' + suffix] = 0 
            df.loc[row, 'GC_STD' + suffix] = 0

### Rest days

In [11]:
df['REST_home'] = -1
df['REST_away'] = -1

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]

        recent = get_last_n_games(team_id, game_date)
        game_count = len(recent)
        
        # overall not the nicest solution, but works
        if game_count > 0:
            last_game = get_last_n_games(team_id, game_date).head(1)     
            try:
                rest_days = (game_date - last_game.iloc[0, 1]).days      # 0 is row, 1 is column (date)
            except:
                rest_days = -1
            
            if rest_days < 0 or rest_days > 21:             # manipulate outliers
                rest_days = 7
            
            df.loc[row, 'REST' + suffix] = rest_days
        else:
            df.loc[row, 'REST' + suffix] = 7    # if now data is available, just one week of rest

## League averages

In [29]:
def get_league_games(league, date):
    games = results[(results['Div'] == league) & (results['Date'] < date) & (results['Date'] > date - datetime.timedelta(730))]
    return games

In [30]:
df['H_GS_AVG' + league_suffix] = 0
df['A_GS_AVG' + league_suffix] = 0
df['H_GS_STD' + league_suffix] = 0
df['A_GS_STD' + league_suffix] = 0
df['H_WIN_PCT' + league_suffix] = 0
df['DRAW_PCT' + league_suffix] = 0

for row, game in results.iterrows():
    league_name = game['Div']
    game_date = game['Date']

    league_games = get_league_games(league_name, game_date)
    game_count = len(league_games)

    # goals scored means
    df.loc[row, 'H_GS_AVG' + league_suffix] = league_games[league_games['Div'] == league_name]['FTHG'].mean()
    df.loc[row, 'A_GS_AVG' + league_suffix] = league_games[league_games['Div'] == league_name]['FTAG'].mean()
    
    # goals scored standard deviations
    df.loc[row, 'H_GS_STD' + league_suffix] = league_games[league_games['Div'] == league_name]['FTHG'].std()
    df.loc[row, 'A_GS_STD' + league_suffix] = league_games[league_games['Div'] == league_name]['FTAG'].std()
   
    home_games_won = len(league_games[(league_games['Div'] == league_name) & (league_games['FTR'] == 1)])
    games_drawn = len(league_games[(league_games['Div'] == league_name) & (league_games['FTR'] == 0)])
   
    if game_count > 0:
        df.loc[row, 'H_WIN_PCT' + league_suffix] = home_games_won / game_count
        df.loc[row, 'DRAW_PCT' + league_suffix] = games_drawn / game_count
    else:
        df.loc[row, 'H_WIN_PCT' + league_suffix] = 0
        df.loc[row, 'DRAW_PCT' + league_suffix] = 0

### Other league features

In [31]:
df['TEAM_CNT' + league_suffix] = 0
df['GD_STD' + league_suffix] = 0
df['RND_CNT' + league_suffix] = 0
for row, game in results.iterrows():
    league_name = game['Div']
    game_date = game['Date']
    
    # all leagues consist of 20 teams, except the Bundesliga (D1)
    if league_name == 'D1':
        df.loc[row, 'TEAM_CNT' + league_suffix] = 18
        df.loc[row, 'RND_CNT' + league_suffix] = 34
    else:
        df.loc[row, 'TEAM_CNT' + league_suffix] = 20
        df.loc[row, 'RND_CNT' + league_suffix] = 38
        
    games = get_league_games(league_name, game_date)
    
    # goal difference standard deviation
    df.loc[row, 'GD_STD' + league_suffix] = abs(games['FTHG'] - games['FTAG']).std() # muss hier wirklich abs() hin?

## Pi-ratings

### Functions

In [37]:
# Constant and error function (called psi) from the pi-ratings paper by Constantinou and Fenton
c = 3
def psi(error):
    return c * np.log10(1 + error)

def psi_h(error, exp_gd, actual_gd):
    if (exp_gd - actual_gd) < 0:
        return psi(error)
    else:
        return -1 * psi(error)

def psi_a(error, exp_gd, actual_gd):
    if (exp_gd - actual_gd) > 0:
        return psi(error)
    else:
        return -1 * psi(error)
    

# Constant and goal difference prediction function
b = 10
def predict_gd(rating):
    if rating >= 0:
        return np.power(b, np.abs(rating) / c) - 1
    else:
        return -(np.power(b, np.abs(rating) / c) - 1)

'''
Returns a dict with keys H_RTG_home, H_RTG_away, A_RTG_away, A_RTG_away
Used for one game!

cur_ratings: input in the same form as the output (dict with 4 elems); current ratings
'''
def update_pi_ratings(cur_ratings, actual_gd):
    H_RTG_home, H_RTG_away, A_RTG_home, A_RTG_away = cur_ratings['H_RTG_home'], cur_ratings['H_RTG_away'], cur_ratings['A_RTG_home'], cur_ratings['A_RTG_away']
    
    exp_gd = predict_gd(H_RTG_home) - predict_gd(A_RTG_away)    # Calculate expected goal difference
    
    error = np.abs(actual_gd - exp_gd)
    
    # The two learning rates from the pi-ratings paper (may have to be adjusted)
    lr_lambda = 0.1
    lr_gamma = 0.3
    
    # Update the pi-ratings
    new_H_RTG_home = H_RTG_home + psi_h(error, exp_gd, actual_gd) * lr_lambda 
    new_H_RTG_away = H_RTG_away + (new_H_RTG_home - H_RTG_home) * lr_gamma
    
    new_A_RTG_away = A_RTG_away + psi_a(error, exp_gd, actual_gd) * lr_lambda
    new_A_RTG_home = A_RTG_home + (new_A_RTG_away - A_RTG_away) * lr_gamma
    
    new_ratings = {'H_RTG_home': new_H_RTG_home, 'H_RTG_away': new_H_RTG_away, 'A_RTG_away': new_A_RTG_away, 'A_RTG_home': new_A_RTG_home}
    
    return new_ratings

In [7]:
update_pi_ratings({'H_RTG_home': 1.6, 'H_RTG_away': 0.4, 'A_RTG_home': 0.3, 'A_RTG_away': -1.2}, 3)

{'H_RTG_home': 1.5145736713398419,
 'H_RTG_away': 0.37437210140195254,
 'A_RTG_away': -1.1145736713398415,
 'A_RTG_home': 0.3256278985980475}

In [8]:
predict_gd(-1.5)

-2.1622776601683795

### Implementation into dataframe

In [9]:
# Returns the ID of the next game of a given team.
def get_next_game_id(team_id, date):
    games = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] > date)].sort_values(by='Date')
    
    if len(games) > 0:
        return games.index[0]    # [0] refers to the first element of the array returned by .index here
    else:
        print("LAST GAME:", date, team_id)
        return None

In [10]:
get_next_game_id(31, datetime.date(2017, 4, 2))

2577

In [47]:
df['H_RTG_home'] = 0
df['A_RTG_home'] = 0
df['H_RTG_away'] = 0
df['A_RTG_away'] = 0

In [ ]:
for row, game in results.iterrows():
    league_name = game['Div']
    game_date = game['Date']    
    
    # Get current pi ratings from dataframe
    cur_ratings = {
        'H_RTG_home': df.loc[row, 'H_RTG_home'],
        'H_RTG_away': df.loc[row, 'H_RTG_away'],
        'A_RTG_home': df.loc[row, 'A_RTG_home'],
        'A_RTG_away': df.loc[row, 'A_RTG_away']
    }
    
    actual_gd = game['FTHG'] - game['FTAG']
    
    new_ratings = update_pi_ratings(cur_ratings, actual_gd)
    
    # Update all pi ratings for the next game (if it wasn't a teams last game)

    next_game_id_H = get_next_game_id(game['HomeID'], game_date)
    next_game_id_A = get_next_game_id(game['AwayID'], game_date)
    
    if next_game_id_H != None:
        df.loc[next_game_id_H, 'H_RTG_home'] = new_ratings['H_RTG_home']
        df.loc[next_game_id_H, 'H_RTG_away'] = new_ratings['H_RTG_away']
    
    if next_game_id_A != None:
        df.loc[next_game_id_A, 'A_RTG_home'] = new_ratings['A_RTG_home']
        df.loc[next_game_id_A, 'A_RTG_away'] = new_ratings['A_RTG_away']
    
    # TODO: Gucken ob die Indizes passen oder ob da kleine errors drin sind die alles durcheinander wirbeln
    

### Expected goal difference for the match

In [43]:
for row, game in results.iterrows():
    exp_gd_match = predict_gd(df.loc[row, 'H_RTG_home']) - predict_gd(df.loc[row, 'A_RTG_away'])
    df.loc[row, 'EGD'] = exp_gd_match

## PageRank

To save computation time, we don't calculate an entire matrix every time. Instead, we just calculate the average points of the two given teams against each other from the last two years (like described in the Hubacek paper). 

In [20]:
# We initialize it with 1 point per game as the default value.
df['EPTS_PR_home'] = 1
df['EPTS_PR_away'] = 1

for row, game in results.iterrows():
    home_id = game['HomeID']
    away_id = game['AwayID']
    
    game_date = game['Date']

    hist_games = get_historical_games(home_id, game_date)  # Could also use away_id here, it doesn't matter

    # Previous matches of the two current teams
    previous_matches = hist_games[((home_id == hist_games['HomeID']) | (home_id == hist_games['AwayID']))
              & ((away_id == hist_games['HomeID']) | (away_id == hist_games['AwayID']))]
    
    game_count = len(previous_matches)
    
    if game_count == 0:
        continue              # Avoid division by zero by just letting the PR for both teams be 1
    
    # Number of wins of the current(!) home team in previous_matches
    win_count_H = len(previous_matches[((previous_matches['HomeID'] == home_id) & previous_matches['FTR'] == 1)
                                      |((previous_matches['AwayID'] == home_id) & previous_matches['FTR'] == 2)])
    
    # Number of draws in previous_matches
    draw_count = len(previous_matches[previous_matches['FTR'] == 0])
    
    # Number of wins of the current(!) away team in previous_matches
    win_count_A = len(previous_matches[((previous_matches['HomeID'] == away_id) & previous_matches['FTR'] == 1)
                                      |((previous_matches['AwayID'] == away_id) & previous_matches['FTR'] == 2)])
    
    df.loc[row, 'EPTS_PR_home'] = (3 * win_count_H + draw_count) / game_count
    df.loc[row, 'EPTS_PR_away'] = (3 * win_count_A + draw_count) / game_count

# Implementing own features

**In the following section, some of my own feature ideas are implemented into the dataframe.**

These can be based on historical data, current form data, league data or whatever else.

## Red Cards

Here, we accumulate the red cards each team collected in the last three games. Missing a player due to a red card can be a potentially important factor.

In [22]:
df['RED_home'] = 0
df['RED_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']
        
        recent = get_last_n_games(team_id, game_date, n=3)
        
        suffix = suffixes[id_name]
        
        df.loc[row, 'RED' + suffix] = recent['HR'].sum()

## Difference between own shots and shots conceded (historical)

In [42]:
df['H_ST_home'] = 0
df['A_ST_home'] = 0
df['H_ST_away'] = 0
df['A_ST_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        hist_games = get_historical_games(team_id, game_date)
        
        home_shots_mean = hist_games[hist_games['HomeID'] == team_id]['HS'].mean()
        away_shots_mean = hist_games[hist_games['AwayID'] == team_id]['AS'].mean()
        
        if np.isnan(home_shots_mean):
            home_shots_mean = 0       # TODO: hier besseren filler value suchen
            
        if np.isnan(away_shots_mean):
            away_shots_mean = 0       # TODO: hier besseren filler value suchen
            
        df.loc[row, 'H_ST' + suffixes[id_name]] = home_shots_mean
        df.loc[row, 'A_ST' + suffixes[id_name]] = away_shots_mean

## Match importance (similar to Hubacek)

### Season

In [77]:
# Returns the season corresponding to a certain date.
def get_season_from_date(date):
    for i in range(9):
        if date < datetime.date(2010 + i+1, 7, 15):
            return i+1

In [78]:
# To build up tables, there needs to be a cut where one season ends and the next begins.
# This is the 17th of July every year here.
df['season'] = 0

for row, game in results.iterrows():
    game_date = game['Date']
    #print(get_season_from_date(game_date))
    df.loc[row, 'season'] = get_season_from_date(game_date)

### Round (1-34 / 1-38)

In [79]:
df['RND'] = 0

prev_season = 1
cur_matchday = 1

modulo_add = 1

# We need to add two missing rows in order to keep the indices in the right order
# They can and should be removed later!
excepted = False
try: 
    df.loc[6969, 'HomeID']
except:
    df.loc[6969] = pd.Series()
    excepted = True
try:
    df.loc[8477, 'HomeID']
except:
    df.loc[8477] = pd.Series()
    excepted = True

df = df.sort_index()
    
if excepted:
    df.index = range(len(df.index))

for i in range(len(df.index)):
    try:
        game_season = df.loc[i, 'season']
        
        if game_season != prev_season:       # check if new season has begun
            cur_matchday = 1

        matches_per_md = int(np.round(df.loc[i, 'TEAM_CNT_league'] / 2))
    
        df.loc[i, 'RND'] = cur_matchday

        # Change modulo_add for the switch from Germany (9 games per matchday) to Italy (10 games per matchday).
        # Used to avoid getting bugs in current matchday.
        if i == 6174:
            modulo_add = -3
            
        if (i + modulo_add) % matches_per_md == 0:      # check if last game of a matchday has been played
            cur_matchday += 1

        prev_season = game_season
        
    except:
        continue

<ipython-input-79-5441dfb28a23>:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[6969] = pd.Series()
<ipython-input-79-5441dfb28a23>:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[8477] = pd.Series()


### Current standings

In [ ]:
# Gets all games of a team that were played before a specific date in the current season
def get_team_season_games(team_id, date, season):
    games_by_team = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] < date) & (results[])]
    return games_by_team

In [32]:
get_last_n_games(8, datetime.date(2010,8,22), n=1, df_mode=True)

,HomeID,AwayID,FTHG,FTAG,FTR,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,H_WIN_PCT_away,...,RED_away,H_ST_home,H_ST_away,A_ST_home,A_ST_away,season,RND,Date,PTS_AFTER_GAME_home,PTS_AFTER_GAME_away
8,8.0,10.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2010-08-15,1,1


In [50]:
# Points after the current game
df['PTS_AFTER_GAME_home'] = 0
df['PTS_AFTER_GAME_away'] = 0

for row, game in df.iterrows():
    #print(row)
    # Game result
    game_res = game['FTR']
    game_date = game['Date']
    
    # Get previous points of both teams (0 if this game is part of the first matchday)
    if game['RND'] == 1:
        prev_points_home = 0
        prev_points_away = 0
    else:
        # Last game played by the home team
        last_game_home = get_last_n_games(game['HomeID'], game_date, n=1, df_mode=True)
        # Last game played by the away team
        last_game_away = get_last_n_games(game['AwayID'], game_date, n=1, df_mode=True)
        
        #print(last_game_home.iloc[0])
        print(row)
        
        # Get points from after the teams' last games
        if game['HomeID'] == last_game_home.iloc[0]['HomeID']:
            prev_points_home = last_game_home.iloc[0]['PTS_AFTER_GAME_home']
        elif game['HomeID'] == last_game_home.iloc[0]['AwayID']:
            prev_points_home = last_game_home.iloc[0]['PTS_AFTER_GAME_away']
        
        if game['AwayID'] == last_game_away.iloc[0]['HomeID']:
            prev_points_away = last_game_away.iloc[0]['PTS_AFTER_GAME_home']
        elif game['AwayID'] == last_game_away.iloc[0]['AwayID']:
            prev_points_away = last_game_away.iloc[0]['PTS_AFTER_GAME_away']
        
    if game_res == 1:      # Home win
        df.loc[row, 'PTS_AFTER_GAME_home'] = prev_points_home + 3
    elif game_res == 0:    # Draw
        df.loc[row, 'PTS_AFTER_GAME_home'] = prev_points_home + 1
        df.loc[row, 'PTS_AFTER_GAME_away'] = prev_points_away + 1
    elif game_res == 2:    # Away win
        df.loc[row, 'PTS_AFTER_GAME_away'] = prev_points_away + 3

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
28

1918
1919


IndexError: single positional indexer is out-of-bounds

In [80]:
df[1895:1922]

,HomeID,AwayID,FTHG,FTAG,FTR,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,H_WIN_PCT_away,...,EPTS_PR_away,RED_home,RED_away,H_ST_home,H_ST_away,A_ST_home,A_ST_away,Date,season,RND
1895,26.0,9.0,0.0,0.0,0.0,0.324324,0.216216,0.157895,0.236842,0.605263,...,2.000000,0.0,0.0,12.783784,15.342105,9.605263,12.000000,2015-05-24,5.0,38.0
1896,28.0,20.0,5.0,1.0,1.0,0.333333,0.277778,0.210526,0.157895,0.315789,...,3.000000,0.0,0.0,14.055556,16.210526,9.578947,11.500000,2015-05-24,5.0,38.0
1897,18.0,24.0,2.0,0.0,1.0,0.810811,0.108108,0.526316,0.210526,0.500000,...,0.333333,0.0,0.0,19.540541,15.447368,15.868421,11.810811,2015-05-24,5.0,38.0
1898,17.0,15.0,2.0,0.0,1.0,0.378378,0.216216,0.263158,0.131579,0.421053,...,1.333333,0.0,0.0,15.891892,13.605263,11.605263,10.324324,2015-05-24,5.0,38.0
1899,13.0,8.0,6.0,1.0,1.0,0.513514,0.243243,0.210526,0.289474,0.684211,...,2.000000,0.0,0.0,14.378378,18.473684,9.973684,14.189189,2015-05-24,5.0,38.0
1900,30.0,0.0,0.0,1.0,2.0,0.000000,0.000000,0.000000,0.000000,0.289474,...,1.000000,0.0,0.0,0.000000,11.973684,0.000000,10.236842,2015-08-08,6.0,1.0
1901,3.0,21.0,2.0,2.0,0.0,0.789474,0.184211,0.552632,0.236842,0.394737,...,0.000000,0.0,0.0,18.078947,12.684211,14.973684,11.421053,2015-08-08,6.0,1.0
1902,12.0,31.0,2.0,2.0,0.0,0.526316,0.263158,0.342105,0.263158,0.000000,...,1.000000,0.0,0.0,15.315789,0.000000,12.157895,0.000000,2015-08-08,6.0,1.0
1903,28.0,4.0,4.0,2.0,1.0,0.368421,0.263158,0.210526,0.157895,0.236842,...,1.000000,0.0,0.0,14.473684,13.236842,9.578947,10.289474,2015-08-08,6.0,1.0
1904,9.0,5.0,1.0,0.0,1.0,0.605263,0.131579,0.421053,0.315789,0.552632,...,0.500000,0.0,0.0,15.342105,15.500000,11.868421,13.657895,2015-08-08,6.0,1.0


In [84]:
df.sample(10)

,HomeID,AwayID,FTHG,FTAG,FTR,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,H_WIN_PCT_away,...,EPTS_PR_away,RED_home,RED_away,H_ST_home,H_ST_away,A_ST_home,A_ST_away,Date,season,RND
4258,42.0,37.0,5.0,1.0,1.0,0.735294,0.147059,0.515152,0.303030,0.352941,...,0.0,0.0,0.0,17.911765,14.176471,14.545455,10.000000,2013-03-16,3.0,26.0
10263,116.0,111.0,0.0,3.0,2.0,0.428571,0.285714,0.214286,0.000000,0.484848,...,0.0,2.0,1.0,12.714286,16.333333,12.785714,12.151515,2012-03-26,2.0,29.0
240,19.0,15.0,1.0,3.0,2.0,0.300000,0.200000,0.384615,0.153846,0.250000,...,1.0,0.0,0.0,13.700000,12.583333,10.384615,10.333333,2011-02-02,1.0,25.0
10901,99.0,120.0,4.0,2.0,1.0,0.358974,0.307692,0.189189,0.297297,0.333333,...,1.0,1.0,0.0,12.512821,11.481481,10.189189,8.888889,2013-12-22,4.0,17.0
823,15.0,10.0,1.0,3.0,2.0,0.333333,0.277778,0.157895,0.368421,0.578947,...,1.5,1.0,0.0,13.277778,16.131579,11.157895,14.108108,2012-10-06,3.0,7.0
2231,30.0,8.0,1.0,2.0,2.0,0.312500,0.250000,0.352941,0.235294,0.472222,...,3.0,0.0,0.0,12.500000,17.833333,11.588235,14.108108,2016-04-17,6.0,34.0
2988,24.0,3.0,2.0,3.0,2.0,0.297297,0.351351,0.263158,0.289474,0.692308,...,2.0,1.0,0.0,14.891892,17.410256,12.184211,13.567568,2018-04-14,8.0,33.0
11164,121.0,119.0,0.0,1.0,2.0,0.285714,0.428571,0.181818,0.227273,0.342105,...,0.0,0.0,0.0,11.476190,13.605263,9.863636,11.052632,2014-09-26,5.0,6.0
3932,42.0,39.0,3.0,1.0,1.0,0.740741,0.185185,0.620690,0.172414,0.642857,...,1.0,0.0,1.0,18.148148,12.964286,15.034483,9.678571,2012-02-26,2.0,23.0
5173,38.0,54.0,2.0,0.0,1.0,0.314286,0.314286,0.205882,0.176471,0.411765,...,2.0,0.0,0.0,12.285714,10.323529,10.882353,8.571429,2016-03-06,6.0,25.0


In [18]:
df.columns

Index(['HomeID', 'AwayID', 'FTHG', 'FTAG', 'FTR', 'H_WIN_PCT_home',
       'H_DRAW_PCT_home', 'A_WIN_PCT_home', 'A_DRAW_PCT_home',
       'H_WIN_PCT_away', 'H_DRAW_PCT_away', 'A_WIN_PCT_away',
       'A_DRAW_PCT_away', 'H_GS_AVG_home', 'H_GC_AVG_home', 'A_GS_AVG_home',
       'A_GC_AVG_home', 'H_GS_AVG_away', 'H_GC_AVG_away', 'A_GS_AVG_away',
       'A_GC_AVG_away', 'H_GS_STD_home', 'H_GC_STD_home', 'A_GS_STD_home',
       'A_GC_STD_home', 'H_GS_STD_away', 'H_GC_STD_away', 'A_GS_STD_away',
       'A_GC_STD_away', 'WIN_PCT_home', 'WIN_PCT_away', 'DRAW_PCT_home',
       'DRAW_PCT_away', 'GS_AVG_home', 'GC_AVG_home', 'GS_AVG_away',
       'GC_AVG_away', 'GS_STD_home', 'GC_STD_home', 'GS_STD_away',
       'GC_STD_away', 'REST_home', 'REST_away', 'H_GS_AVG_league',
       'A_GS_AVG_league', 'H_GS_STD_league', 'A_GS_STD_league',
       'H_WIN_PCT_league', 'DRAW_PCT_league', 'TEAM_CNT_league',
       'GD_STD_league', 'RND_CNT_league', 'H_RTG_home', 'A_RTG_home',
       'H_RTG_away', 'A_RTG_aw

# Save dataframe as .pkl

In [324]:
df.to_pickle('feature_frame.pkl')